# *IceFlow & icepyx*: Altimetry Time Series Tutorial
### Earthdata Webinar - April 2021</b>

<b>Authors:</b><br />
<span style="font-size:larger;">Jessica Scheick</span>, *University of New Hampshire*, Durham, New Hampshire<br />
<span style="font-size:larger;">Nicholas Kotlinski & Amy Steiker</span>, *NASA National Snow and Ice Data Center DAAC*, Boulder, Colorado, USA

---

#### Running this tutorial locally

To run this notebook locally, you must first create a computing environment. The easiest way to do this is by downloading the [`environment.yml`](https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/binder/environment.yml) file from [NSIDC's GitHub Repository of Data Tutorials](https://github.com/nsidc/NSIDC-Data-Tutorials). Then, navigate to the directory where you've stored the file and run:
```
conda env create -n iceflow-env -f environment.yml
activate iceflow-env
```

Then open the notebook using your preferred viewer ([JupyterLab](https://jupyterlab.readthedocs.io/en/stable/getting_started/starting.html) is included in this environment). More detailed instructions on setting up environments with conda can be found [here](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file).

This notebook can be downloaded from [GitHub](https://github.com/nsidc/NSIDC-Data-Tutorials/tree/main/notebooks).

<b>Other needs:</b><br />
* X Markdown on how to run the notebook locally (we might need to adjust the link for where the notebook is).
* IceFlow API examples
* icepyx examples
* Code to configure an ICESat-2 file to match IceFlow data dictionary

### 1. NASA's Earthdata Credentials

To access data using the *IceFlow* library and *icepyx* package, it is necessary to log into [Earthdata Login](https://urs.earthdata.nasa.gov/). To do this, enter your NASA Earthdata credentials in the next step after executing the following code cell.

**Note**: If you don't have NASA Earthdata credentials you will need to register first at the link above. An account is free and available to everyone!

In [ ]:
# This cell will prompt you for your Earthdata username and password. Press
from iceflow.ui import IceFlowUI
client = IceFlowUI()
client.display_credentials()

In [ ]:
# This cell will verify if your credentials are valid. 
# This may take a little while. If it fails, please try again.

authorized = client.authenticate()
if authorized is None:
    print('Earthdata Login not successful')
else:
    print('Earthdata Login successful!')

**Note:** If the output shows "You are logged into NASA Earthdata!", then you are ready to proceed!

---

### 2. Accessing and harmonizing data across missions
#### 2.1. Accessing Data with the *IceFlow* Access Widget
The *IceFlow* access widget is a user interface tool to visualize flightpaths from IceBridge, draw a region of interest, set spatio-temporal parameters and place data orders to the *IceFlow* API and *icepyx* package without the need to writing code.
The output of the operations performed in the widget can be seen in the log window (right-most icon at the bottom of your browser.) 
<img src='./img/log-icons.png'> or by selecting it on the View menu "Show log console"

**Note:** The access widget is currently stateless, so if you change any parameter you will have to redraw your bounding box or polygon.

In [ ]:
# Let's start with the user interface. Using 'horizontal' will add the widget inline.
client.display_map('horizontal', extra_layers=True)

#### 2.2. Accessing data with the *IceFlow* API

In [ ]:
# Small example subset over Jakobshavn glacier:
my_params ={
    'datasets': ['ATM1B', 'GLAH06', 'ILVIS2', 'ATL06'],
    'start': '2008-01-01',
    'end': '2018-12-31',
    'bbox': '-49.787268,69.103805,-49.526401,69.218717',
    'itrf': 'ITRF2014',
    'epoch': '2014.1'
}

# returns a json dictionary, the request parameters and the order's response.
granules_metadata = client.query_cmr(params=my_params)

In [ ]:
orders = client.place_data_orders(params=my_params)
orders

#### Check Order Status
The following cell will show you the status of your data order. You can proceed in the notebook once all orders are "COMPLETE". If you proceed earlier only the completed data orders will be downloaded.

In [ ]:
for order in orders:
    status = client.order_status(order)
    print(order['dataset'], order['id'], status['status'])

#### Download Data
Once all data orders are "COMPLETE", you can proceed downloading the data:

In [ ]:
for order in orders:
    status = client.order_status(order)
    if status['status'] == 'COMPLETE':
        client.download_order(order)

#### 2.3. Downloading ICESat-2 data with *icepyx*
OR What's going on behind the scenes with *icepyx*

In [ ]:
# Begin overview of icepyx including visualzation options

---

### 3. Working with the data
Now that we have downloaded our data, we need to make sure that they are in a common format to do analysis across missions.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
import geopandas as gpd
from iceflow.processing import IceFlowProcessing as ifp

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Pre-IceBridge ATM granule data
preib_gdf = ifp.to_geopandas('data/atm1b_data_2020-11-15T20-05.hdf5')
# ICESat granule data
glas_gdf = ifp.to_geopandas('data/twaties-test-GLAH06-2000-2010.h5')

In [ ]:
# Either add processing for ICESat-2 to processing.py or do it in-line here
# ICESat-2 granule data
#is2_gdf = ifp.to_geopandas('data/processed_ATL06_20181015100401_02560110_003_01.h5')

In [ ]:
# first, let's see what's in the harmonized dataframe and its shape.
display(preib_gdf.head(), preib_gdf.shape)

In [ ]:
# we do the same for the ICESat/GLAS dataframe
display(glas_gdf.head(), glas_gdf.shape)

---

Use code and example from https://nbviewer.jupyter.org/github/nicholas-kotlinski/2020_ICESat-2_Hackweek_Tutorials/blob/master/05.Geospatial_Analysis/shean_ICESat-2_hackweek_tutorial_GeospatialAnalysis_rendered.ipynb

## [Draft notebook for April 2021 Earthdata Webinar]

#### Learning Objectives (tbd)
* (goal from discussion) Introduce audience to the “Ice cubed” (ICESat/OIB/ICESat-2) missions and products
* (goal from discussion Promote icepyx and IceFlow as a way to address the need to access/harmonize multiple data products/resolutions/formats/structure


#### Webinar Outline
* Intro of harmonization challenge
    - Moving one level deeper into the need for time series analysis across disparate platforms
* Intro to the altimetry missions
    - Use the IceFlow intro notebook to guide this
    - Highlight the impressive time series
* Overview of IceFlow and icepyx
    - Audience and use cases of these tools
    - Icepyx genesis, open science and community development principles
        - working on that balance between community development and end use
    - IceFlow overview
    - How do these tools address challenges
        - IceFlow aims to make it easier to harmonize OIB/ICESat data with ICESat-2 using backend IceFlow API
* Use Case / Application
    - Leverage ICESat-2 hackweek topics
    - https://github.com/ICESAT-2HackWeek/geospatial-analysis/blob/master/shean_ICESat-2_hackweek_tutorial_GeospatialAnalysis.ipynb
* Short navigation of nsidc site, how to get to tools, notebook
    - Have Jennifer post links while we talk.
* Demos
    - **Use this notebook to pulls pieces of iceflow / icepyx using a specific science application**